# Asymptotic approximations

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sandialabs/Polymers/main?labpath=docs%2Fsource%2F%2Fphysics%2Fsingle_chain%2Ffjc%2Fthermodynamics%2Fmodified_canonical%2Fexample_asymptotic.ipynb)

This example demonstrates the validity of the asymptotic approaches of approximating the thermodynamics of the freely-jointed chain (FJC) model in the modified canonical ensemble. To start, import and create an instance of the FJC model in the modified canonical ensemble:

In [ ]:
from polymers import physics
fjc = physics.single_chain.fjc.thermodynamics.modified_canonical.FJC(8, 1, 1)

## Strong potential

For sufficiently strong potentials, the modified canonical ensemble can be accurately approximated using the reference system (the isometric ensemble) and an asymptotic correction. For example, the nondimensional force $\eta$ as a function of the nondimensional potential distance $\gamma$ is approximated as

$$
  \eta(\gamma) \sim \eta_0(\gamma) - \frac{1}{\varpi}\left[\eta_0(\gamma)\,\frac{\partial\eta_0(\gamma)}{\partial\gamma} - \frac{1}{2N_b}\frac{\partial^2\eta_0(\gamma)}{\partial\gamma^2}\right]\quad\text{for }\varpi\gg 1,
$$

where $\varpi$ is the nondimensional potential stiffness, $N_b$ is the number of links, and the nondimensional force $\eta_0$ for the freely-jointed chain model in the isometric ensemble as a function of the nondimensional end-to-end length per link $\gamma'$ is given by

$$
  \eta_0(\gamma') = \frac{1}{N_b\gamma'} + \left(\frac{1}{2} - \frac{1}{N_b}\right)\frac{\sum_{s=0}^{s_\mathrm{max}}(-1)^s\binom{N_b}{s}\left(\frac{1-\gamma'}{2} - \frac{s}{N_b}\right)^{N_b - 3}}{\sum_{s=0}^{s_\mathrm{max}}(-1)^s\binom{N_b}{s}\left(\frac{1-\gamma'}{2} - \frac{s}{N_b}\right)^{N_b - 2}}\quad\text{where }s_\mathrm{max}\leq N_b m\leq s_\mathrm{max}+1.
$$

In contrast, the exact relation for $\eta(\gamma)$ is obtained by numerically evaluating

$$
  \eta(\gamma) = -\frac{1}{N_b}\frac{\partial}{\partial\gamma}\,\ln\left[\iiint Q_0(\boldsymbol{\gamma}') e^{-\tfrac{\varpi}{2} N_b^2\left(\boldsymbol{\gamma} - \boldsymbol{\gamma}'\right)^2} \,d^3\boldsymbol{\gamma}'\right],
$$

where $Q_0$ is the partition function in the isometric ensemble. This exact relation is plotted below along with the asymptotic relation while varying $\varpi$, the nondimensional potential stiffness. As $\varpi$ increases, the asymptotic approach appears to do increasingly well, but always eventually fails as $\gamma$ becomes large. This is because the asymptotic approach in this case is completely invalid for larger values of $\gamma$ (approaching 1), where the massive associated entropic force would cause the end of the chain to leave the bottom of the potential well, even though it may be quite strong.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
gamma = np.linspace(1e-3, 99e-2, 100)
for varpi in [1e1, 5e1, 5e2]:
    eta = fjc.nondimensional_force(gamma, varpi)
    line = plt.plot(gamma, eta, label=r'$\varpi=$' + str(varpi))
    eta_asymptotic = fjc.asymptotic.strong_potential.nondimensional_force(gamma, varpi)
    plt.plot(gamma, eta_asymptotic, ':', color=line[0].get_color())
plt.legend()
plt.xlim([0, 1])
plt.ylim([0, 14])
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$\eta$')
plt.show()

## Weak potential

For sufficiently distant potentials, the modified canonical ensemble can be accurately approximated using the reference system (the isotensional ensemble) and an asymptotic correction. The potential is considered sufficiently distant when the length of center of the potential well to the end of the chain experiencing it is much larger than the expected end-to-end length of the chain. This distance is only typically possible with weak potentials.